In [1]:
import pandas as pd
import numpy as np

In [2]:
cep = pd.read_csv("base_data_CEP.csv")

In [3]:
cep.head()

,fisc_wk_id,fisc_wk_strt_dt,mkt_lvl,rgn_shrt_nm,co_nbr,co_shrt_nm,bus_ctr_nm,itm_grp_nm,attr_grp_nm,prc_src_typ_cd,cust_nbr,cuisine_lvl_1_desc,ttl_cases,ttl_sales,ttl_lines,ttl_grs_prft
0,202218,2021-10-31,North,New England,8,NORTHERN NEW ENGLAND,BAKERY,BREADS,BREADS/ROLLS,CEP,136804,Specialty American,2.0,55.70,1,11.259
1,202218,2021-10-31,North,New England,8,NORTHERN NEW ENGLAND,BAKERY,BREADS,BREADS/ROLLS,CEP,168680,Mainstream American,5.0,133.45,1,29.699
2,202218,2021-10-31,North,New England,8,NORTHERN NEW ENGLAND,BAKERY,BREADS,BREADS/ROLLS,CEP,193740,Mainstream American,1.0,28.35,1,7.764
3,202218,2021-10-31,North,New England,8,NORTHERN NEW ENGLAND,BAKERY,BREADS,BREADS/ROLLS,CEP,242820,Italian,2.0,80.54,2,23.433
4,202218,2021-10-31,North,New England,8,NORTHERN NEW ENGLAND,BAKERY,BREADS,BREADS/ROLLS,CEP,243345,Bar or Brewery,2.0,62.80,1,6.658


In [4]:
cep.columns

Index(['fisc_wk_id', 'fisc_wk_strt_dt', 'mkt_lvl', 'rgn_shrt_nm', 'co_nbr',
       'co_shrt_nm', 'bus_ctr_nm', 'itm_grp_nm', 'attr_grp_nm',
       'prc_src_typ_cd', 'cust_nbr', 'cuisine_lvl_1_desc', 'ttl_cases',
       'ttl_sales', 'ttl_lines', 'ttl_grs_prft'],
      dtype='object')

In [5]:
cols = ['cust_nbr','fisc_wk_id','ttl_cases','fisc_wk_strt_dt']


cep = cep[cols]

In [6]:
cep['fisc_wk_strt_dt'] = pd.to_datetime(cep['fisc_wk_strt_dt'])

In [7]:
cep["year"] = cep['fisc_wk_strt_dt'].dt.year

cep.drop('fisc_wk_strt_dt', axis = 1, inplace = True)

In [8]:
cep.sort_values(["year",'cust_nbr'], ascending = True, inplace = True)

In [9]:
cep.head()

,cust_nbr,fisc_wk_id,ttl_cases,year
1038,36111,202218,1.0,2021
1165,36111,202218,1.0,2021
2292,36111,202219,1.0,2021
2363,36111,202219,1.0,2021
2386,36111,202219,1.0,2021


In [10]:
grouped = cep.groupby(["cust_nbr","year"])["ttl_cases"].agg(["mean",lambda x: np.percentile(x,20)])

In [11]:
grouped.columns = ["year avg cases","min_20_percentile"]

In [12]:
grouped.reset_index(inplace = True)

In [13]:
cep.head()

,cust_nbr,fisc_wk_id,ttl_cases,year
1038,36111,202218,1.0,2021
1165,36111,202218,1.0,2021
2292,36111,202219,1.0,2021
2363,36111,202219,1.0,2021
2386,36111,202219,1.0,2021


In [14]:
grouped.sort_values(["cust_nbr", "year"], ascending = True).head(10)

,cust_nbr,year,year avg cases,min_20_percentile
0,5785,2023,7.875000,1.6
1,21758,2023,17.000000,17.0
2,36111,2021,1.000000,1.0
3,36111,2022,3.418182,1.0
4,36111,2023,2.909091,1.0
5,36475,2021,2.985075,1.0
6,36475,2022,4.318664,1.0
7,36475,2023,3.253213,1.0
8,56226,2023,2.750000,1.6
9,104505,2022,2.000000,1.0


In [15]:
final = cep.merge(grouped, how = "left", left_on = ["cust_nbr", "year",], right_on =["cust_nbr","year",])

In [16]:
def lth_20(row):
    if  row["ttl_cases"] <= row["min_20_percentile"]:
        return 0
    return 1

In [17]:
final["<= 20th percentile"] = final.apply(lth_20, axis = 1)

In [18]:
final["<= 20th percentile"].value_counts()

0    70965
1    70813
Name: <= 20th percentile, dtype: int64

In [19]:
final.columns

Index(['cust_nbr', 'fisc_wk_id', 'ttl_cases', 'year', 'year avg cases',
       'min_20_percentile', '<= 20th percentile'],
      dtype='object')

In [20]:
final = final[['cust_nbr','year', 'fisc_wk_id', 'ttl_cases', 'year avg cases',
       'min_20_percentile', '<= 20th percentile']]

In [21]:
final.to_csv("20th percentile.csv", index = False)

In [22]:
cep.shape

(141778, 4)

In [23]:
final[final["cust_nbr"] == 36111].head(60)

,cust_nbr,year,fisc_wk_id,ttl_cases,year avg cases,min_20_percentile,<= 20th percentile
0,36111,2021,202218,1.0,1.000000,1.0,0
1,36111,2021,202218,1.0,1.000000,1.0,0
2,36111,2021,202219,1.0,1.000000,1.0,0
3,36111,2021,202219,1.0,1.000000,1.0,0
4,36111,2021,202219,1.0,1.000000,1.0,0
5,36111,2021,202219,1.0,1.000000,1.0,0
6,36111,2021,202219,1.0,1.000000,1.0,0
7,36111,2021,202220,1.0,1.000000,1.0,0
8,36111,2021,202221,1.0,1.000000,1.0,0
8992,36111,2022,202227,2.0,3.418182,1.0,1


In [33]:
final["min_20_percentile"].sort_values(ascending = False)

41694     240.000
108340    120.000
1840      120.000
55804     100.000
53288      80.000
           ...   
127886      0.100
127887      0.100
127888      0.100
127889      0.100
141616      0.066
Name: min_20_percentile, Length: 141778, dtype: float64